In [53]:
import json
import re
import os


# only needed if trailing object error

# for filename in os.listdir('../logs/linux_jonas/vanilla_plotly_logs/'):
#     f = os.path.join('../logs/linux_jonas/vanilla_plotly_logs/', filename)
#     if os.path.isfile(f) & f.endswith(".json"):
#         with open(f, 'r') as r:
#             content = r.read()
#         content = re.sub(r'}\s*{', '},{', content)
#         json_data = json.loads('[' + content + ']')
#         with open(f, 'w') as w:
#             json.dump(json_data, w)

In [54]:
import pandas as pd
import numpy as np
import json
import os

# df = pd.read_json('logs/n100000_s100_t10_everynth.json', orient ='index')
df = pd.DataFrame()


for filename in os.listdir('../logs/linux_jonas/vanilla_plotly_logs/'):
    f = os.path.join('../logs/linux_jonas/vanilla_plotly_logs/', filename)
    if os.path.isfile(f)  & (filename.split('.')[1] == 'json'):
        dft = pd.read_json(f)
        dft['datapoints'] = filename.split('_')[0][1:]
        dft['samples'] = filename.split('_')[1][1:]
        dft['traces'] = filename.split('_')[2][1:]
        dft['sampling algorithm'] = filename.split('_')[3]
        dft['iteration'] = filename.split("_")[4].split('.')[0][-1:]
        df = df.append(dft, ignore_index=True)
        


df['message']= df['message'].str.split('\"').str[-2]
df.drop(df[df['level'].eq('WARNING')].index, inplace=True)
df = df.drop('source', axis=1)
df.reset_index()
df['message']


0                                           100.0%
1             time (visible): 1805.199951171875 ms
2        render time (visible): 827.38818359375 ms
3              time (invisible): 1.220947265625 ms
4                time (full): 1808.047119140625 ms
                           ...                    
52795          time (visible): 261.415771484375 ms
52796      render time (visible): 81.3662109375 ms
52797          time (invisible): 0.520751953125 ms
52798             time (full): 264.547607421875 ms
52799                render time (invisible): None
Name: message, Length: 52800, dtype: object

In [55]:
mask = df['message'].str.contains('%')
percentage = df[mask]
percentage
df.loc[mask, 'percentage'] = percentage['message']
df['percentage'].fillna(method='ffill', inplace=True)
df.drop(percentage.index, inplace=True)
df = df.reset_index()
df[['message', 'percentage','timestamp']]

,message,percentage,timestamp
0,time (visible): 1805.199951171875 ms,100.0%,2023-08-07 17:06:47.462
1,render time (visible): 827.38818359375 ms,100.0%,2023-08-07 17:06:47.463
2,time (invisible): 1.220947265625 ms,100.0%,2023-08-07 17:06:47.464
3,time (full): 1808.047119140625 ms,100.0%,2023-08-07 17:06:47.464
4,render time (invisible): None,100.0%,2023-08-07 17:06:47.464
...,...,...,...
47995,time (visible): 261.415771484375 ms,10.0%,2023-08-08 00:53:52.332
47996,render time (visible): 81.3662109375 ms,10.0%,2023-08-08 00:53:52.332
47997,time (invisible): 0.520751953125 ms,10.0%,2023-08-08 00:53:52.332
47998,time (full): 264.547607421875 ms,10.0%,2023-08-08 00:53:52.333


In [56]:
df['type']= df['message'].str.split().str[0]
df['update'] = df['message'].str.split("(").str[1].str.split(")").str[0]
df['time (ms)'] = df['message'].str.split(":").str[1].str.split().str[0]
df['type'] = df['type'].apply(lambda x: x + ' time' if x == 'render' else x)

In [57]:
df['type'] = df['type'].apply(lambda x: x + ' time' if x == 'render' else x)
df

,index,level,message,timestamp,datapoints,samples,traces,sampling algorithm,iteration,percentage,type,update,time (ms)
0,1,DEBUG,time (visible): 1805.199951171875 ms,2023-08-07 17:06:47.462,10000000,10000,100,everynth,0,100.0%,time,visible,1805.199951171875
1,2,DEBUG,render time (visible): 827.38818359375 ms,2023-08-07 17:06:47.463,10000000,10000,100,everynth,0,100.0%,render time,visible,827.38818359375
2,3,DEBUG,time (invisible): 1.220947265625 ms,2023-08-07 17:06:47.464,10000000,10000,100,everynth,0,100.0%,time,invisible,1.220947265625
3,4,DEBUG,time (full): 1808.047119140625 ms,2023-08-07 17:06:47.464,10000000,10000,100,everynth,0,100.0%,time,full,1808.047119140625
4,5,INFO,render time (invisible): None,2023-08-07 17:06:47.464,10000000,10000,100,everynth,0,100.0%,render time,invisible,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,52795,DEBUG,time (visible): 261.415771484375 ms,2023-08-08 00:53:52.332,100000,5000,50,everynth,9,10.0%,time,visible,261.415771484375
47996,52796,DEBUG,render time (visible): 81.3662109375 ms,2023-08-08 00:53:52.332,100000,5000,50,everynth,9,10.0%,render time,visible,81.3662109375
47997,52797,DEBUG,time (invisible): 0.520751953125 ms,2023-08-08 00:53:52.332,100000,5000,50,everynth,9,10.0%,time,invisible,0.520751953125
47998,52798,DEBUG,time (full): 264.547607421875 ms,2023-08-08 00:53:52.333,100000,5000,50,everynth,9,10.0%,time,full,264.547607421875


In [60]:

# Convert "time (ms)" column to numeric data type
df['time (ms)'] = pd.to_numeric(df['time (ms)'], errors='coerce')
df['traces'] = pd.to_numeric(df['traces'])

# aggregation_functions = {
#     'time (ms)': ['mean', lambda x: np.std(x, ddof=0) if len(x) > 1 else 0, 'var']
# }
aggregation_functions = {
    'time (ms)': ['mean', 'var']
}

filtered_df = df.query('`time (ms)` == `time (ms)`')

grouped_df = df.groupby(['update', 'type', 'percentage', 'datapoints', 'samples', 'traces']).agg(aggregation_functions)
agg_df = grouped_df.reset_index()
agg_df.columns = ['update', 'type', 'percentage', 'datapoints', 'samples', 'traces', 'mean_time', 'variance']
agg_df['std_dev'] = np.sqrt(agg_df["variance"])

In [61]:
agg_df.to_csv('../logs/linux_jonas/vanilla_plotly_logs/processed_logs_vanilla.csv')